In [57]:
# Contar indidencia de "1"s na coluna U
import numpy as np
import pandas as pd

df = pd.read_csv('Classificacao_WHOQOL/Datasets/df_base4.csv')


target = df['U']
df = df.drop(['U'], axis=1)


In [119]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

#Balancear os dados com undersampling
# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(random_state=0)
# df, target = rus.fit_resample(df, target)

X_train, X_, y_train, y_ = train_test_split(df, target,
                                                    train_size=0.80, test_size=0.20)


# Aplicando SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=0)
X_train, y_train = smote.fit_resample(X_train, y_train)


X_test, X_cv, y_test, y_cv = train_test_split(X_, y_,
                                                    train_size=0.60, test_size=0.40)

In [ ]:
tpot = TPOTClassifier(verbosity=2, n_jobs=-1)
tpot.fit(X_train, y_train)
print(tpot.score(X_cv, y_cv))

In [ ]:
# Plotando 

In [21]:
# Aplicando z-score
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_cv = scaler.transform(X_cv)
X_test = scaler.transform(X_test)


In [32]:
# Aplicando SBS
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.base import clone
import matplotlib.pyplot as plt
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

# SVM
from sklearn.svm import SVC
svm = SVC()


sbs = SFS(svm,
          k_features="best",
          forward=True,
          floating=False,
          scoring='accuracy',
          cv=0)

sbs = sbs.fit(X_train, y_train)
sbs.k_feature_names_

# Pega os índices das colunas selecionadas e treina o modelo
cols = list(sbs.k_feature_idx_)
svm.fit(X_train[:, cols], y_train)
y_pred = svm.predict(X_test[:, cols])
print('Acurácia: %.3f' % accuracy_score(y_test, y_pred))




Acurácia: 0.556


In [25]:
# Aplicando o modelo
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
rf.fit(X_train[:, cols], y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [28]:
# Avaliando o modelo
y_pred = rf.predict(X_test[:, cols])
print('Acurácia: %.3f' % accuracy_score(y_test, y_pred))


Acurácia: 0.596
0.5959595959595959


In [156]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Define the neural network model with dropout for regularization
class RobustClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_prob):
        super(RobustClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

# Define hyperparameters
input_size = 92  # Number of input features
hidden_size = 128  # Number of units in the hidden layer
output_size = 1  # Number of output units (1 for binary classification)
learning_rate = 0.001
num_epochs = 100
dropout_prob = 0.5  # Dropout probability for regularization

# Split your data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Standardize your data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Convert your data to NumPy arrays and then to PyTorch tensors
X_train_tensor = torch.Tensor(np.array(X_train))
y_train_tensor = torch.Tensor(np.array(y_train).reshape(-1, 1))
X_val_tensor = torch.Tensor(np.array(X_val))
y_val_tensor = torch.Tensor(np.array(y_val).reshape(-1, 1))
X_cv_tensor = torch.Tensor(np.array(X_cv))
y_cv_tensor = torch.Tensor(np.array(y_cv).reshape(-1, 1))


# Create an instance of the model with dropout
model = RobustClassifier(input_size, hidden_size, output_size, dropout_prob)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop with early stopping based on validation loss
best_val_loss = float('inf')
for epoch in range(num_epochs):
    # Forward pass
    model.train()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validate the model on the validation set
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}')
        predictions = model(torch.Tensor(X_cv_tensor))
        predicted_classes = (predictions > 0.5).float().numpy()
        accuracy = accuracy_score(y_cv_tensor, predicted_classes)
        print(f'Accuracy on CV data: {accuracy * 100:.2f}%')

    # Early stopping based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
    else:
        print("Early stopping as validation loss is not improving.")
        break

    

# Make predictions on the test data
with torch.no_grad():
    model.eval()
    predictions = model(torch.Tensor(X_test))
    predicted_classes = (predictions > 0.5).float().numpy()

# Evaluate the model (e.g., calculate accuracy)
accuracy = accuracy_score(y_test, predicted_classes)
print(f'Accuracy on test data: {accuracy * 100:.2f}%')


ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [53]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 4060'

In [90]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from skorch import NeuralNetClassifier
from skorch.callbacks import Callback
from skorch.dataset import CVSplit
from skorch.helper import EarlyStopping

# Custom callback to track the best validation loss
class ValidLossCallback(Callback):
    def __init__(self):
        super().__init__()
        self.best_valid_loss = np.inf

    def on_epoch_end(self, net, **kwargs):
        valid_loss = net.history[-1, 'valid_loss']
        if valid_loss < self.best_valid_loss:
            self.best_valid_loss = valid_loss
            net.best_valid_loss = valid_loss

# Define the neural network model with dropout for regularization
class RobustClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_prob):
        super(RobustClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_prob)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

# Define hyperparameters
input_size = 92  # Number of input features
output_size = 1  # Number of output units (1 for binary classification)
dropout_prob = 0.5  # Dropout probability for regularization

# Load your data (X_train, y_train, X_cv, y_cv, X_test, y_test)



# Standardize your data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_cv = scaler.transform(X_cv)
X_test = scaler.transform(X_test)


X_train_tensor = torch.Tensor(np.array(X_train))
y_train_tensor = torch.Tensor(np.array(y_train).reshape(-1, 1))
X_cv_tensor = torch.Tensor(np.array(X_cv))
y_cv_tensor = torch.Tensor(np.array(y_cv).reshape(-1, 1))


# Create an instance of the custom model with dropout
model = NeuralNetClassifier(
    RobustClassifier,
    module__input_size=input_size,
    module__hidden_size=64,  # Initial hidden size
    module__output_size=output_size,
    module__dropout_prob=dropout_prob,
    optimizer=optim.Adam,
    optimizer__lr=0.001,
    criterion=nn.BCELoss,
    callbacks=[('valid_loss', ValidLossCallback()), ('early_stop', EarlyStopping(patience=10))],
    max_epochs=100,
    batch_size=64,
    train_split=CVSplit(stratified=True, random_state=42),  # Enable train/validation split
)

# Define hyperparameters to search
param_grid = {
    'module__hidden_size': [64, 128, 256],
    'optimizer__lr': [0.001, 0.01, 0.1]
}

# Use cross-validation for hyperparameter tuning
cv_splits = 5  # Number of cross-validation folds
skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=42)

# Create a GridSearchCV object for hyperparameter tuning with skorch
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=skf, verbose=2)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best hyperparameters from the grid search
best_params = grid_search.best_params_
best_hidden_size = best_params['module__hidden_size']
best_learning_rate = best_params['optimizer__lr']

print(f'Best hyperparameters: Hidden Size = {best_hidden_size}, Learning Rate = {best_learning_rate}')

# Create a new instance of the custom model with the best hyperparameters
best_model = NeuralNetClassifier(
    RobustClassifier,
    module__input_size=input_size,
    module__hidden_size=best_hidden_size,
    module__output_size=output_size,
    module__dropout_prob=dropout_prob,
    optimizer=optim.Adam,
    optimizer__lr=best_learning_rate,
    criterion=nn.BCELoss,
    callbacks=[('valid_loss', ValidLossCallback()), ('early_stop', EarlyStopping(patience=10))],
    max_epochs=100,
    batch_size=64,
    train_split=CVSplit(stratified=True, random_state=42),  # Enable train/validation split
)

# Fit the best model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test data using the best model
predicted_classes = best_model.predict(X_test)

# Evaluate the model (e.g., calculate accuracy)
accuracy = accuracy_score(y_test, predicted_classes)
print(f'Accuracy on test data: {accuracy * 100:.2f}%')

ImportError: cannot import name 'CVSplit' from 'skorch.dataset' (/home/apo-pc/anaconda3/envs/bic/lib/python3.11/site-packages/skorch/dataset.py)

In [74]:
X_train.type()

AttributeError: 'numpy.ndarray' object has no attribute 'type'